[Ensemble - RandomForest & ExtraTree]
- 배깅 방식: 중복 허용, 랜덤 샘플, 동일 모델(Decision Tree)
    * 대표 알고리즘: RandomForest
- 페이스트 방식: 중복 X, 랜덤 샘플, 동일 모델(Decision Tree)
    * 대표 알고리즘: EXtraTree

[와인 분류]
- 2개 종류(0 or 1) 분류

1) 모듈 로딩 및 데이터 준비

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_file='../data/wine.csv'

In [3]:
wine_df=pd.read_csv(data_file)

In [4]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [5]:
wine_df.head(3)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0


In [6]:
wine_df['class'].value_counts()

class
1.0    4898
0.0    1599
Name: count, dtype: int64

In [7]:
wine_df.describe()

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


2) 학습 준비

In [8]:
feature_df=wine_df[wine_df.columns[:-1]]
target_sr=wine_df[wine_df.columns[-1]]

print('[feature_df]',feature_df.shape, '[target_sr]', target_sr.shape)

[feature_df] (6497, 3) [target_sr] (6497,)


In [9]:
#데이터셋 분리
from sklearn.model_selection import train_test_split

In [10]:
x_train,x_test,y_train,y_test=train_test_split(feature_df,target_sr,
                                                random_state=1,
                                                test_size=0.2,
                                                stratify=target_sr)

In [11]:
print('x_train:', x_train.shape, 'y_train:', y_train.shape)
print('x_test:',x_test.shape, 'y_test:',y_test.shape)

x_train: (5197, 3) y_train: (5197,)
x_test: (1300, 3) y_test: (1300,)


3) 학습 진행
- 학습 방법: 지도학습 중 분류
- 알고리즘: Ensemble-bagging(RandomForestClassifier)

In [15]:
from sklearn.ensemble import ExtraTreesClassifier

In [16]:
#인스턴스 생성
lf_model=ExtraTreesClassifier(random_state=7)

#학습 진행
lf_model.fit(x_train,y_train)

ExtraTreesClassifier(random_state=7)

In [17]:
#모델 파라미터 확인
print('[classes]',lf_model.classes_)
print('[n_classes]',lf_model.n_classes_,'개')
print()
print('[features names]',lf_model.feature_names_in_)
print('[n features in]',lf_model.n_features_in_,'개')
print('[features importance]',lf_model.feature_importances_)

[classes] [0. 1.]
[n_classes] 2 개

[features names] ['alcohol' 'sugar' 'pH']
[n features in] 3 개
[features importance] [0.18992806 0.53030305 0.2797689 ]


In [18]:
print('[best estimator]',lf_model.estimator_)

for i in lf_model.estimators_:
    print(i)

[best estimator] ExtraTreeClassifier()
ExtraTreeClassifier(random_state=327741615)
ExtraTreeClassifier(random_state=976413892)
ExtraTreeClassifier(random_state=1202242073)
ExtraTreeClassifier(random_state=1369975286)
ExtraTreeClassifier(random_state=1882953283)
ExtraTreeClassifier(random_state=2053951699)
ExtraTreeClassifier(random_state=959775639)
ExtraTreeClassifier(random_state=1956722279)
ExtraTreeClassifier(random_state=2052949340)
ExtraTreeClassifier(random_state=1322904761)
ExtraTreeClassifier(random_state=165338510)
ExtraTreeClassifier(random_state=1133316631)
ExtraTreeClassifier(random_state=4812360)
ExtraTreeClassifier(random_state=372560217)
ExtraTreeClassifier(random_state=309457262)
ExtraTreeClassifier(random_state=1801189930)
ExtraTreeClassifier(random_state=1152936666)
ExtraTreeClassifier(random_state=68334472)
ExtraTreeClassifier(random_state=2146978983)
ExtraTreeClassifier(random_state=119248870)
ExtraTreeClassifier(random_state=769786948)
ExtraTreeClassifier(random_st

4) 성능 평가

In [19]:
train_score=lf_model.score(x_train,y_train)
test_score=lf_model.score(x_test,y_test)

In [20]:
print('[train_score]',train_score)
print('[test_score]',test_score)

[train_score] 0.9973061381566288
[test_score] 0.8961538461538462


5) 튜닝
- RandomizedSearchCV 하이퍼 파라미터 최적화 클래스
    * 범위가 넓은 하이퍼 파라미터 설정에 좋음
    * 지정된 범위에서 지정된 횟수만큼 하이퍼 파라미터 추출하여 조합 진행

In [21]:
from sklearn.model_selection import RandomizedSearchCV

In [22]:
params={'max_depth':range(2,16),
        'min_samples_leaf': range(5,16),
        'criterion':['gini','entropy','log_loss']}

In [29]:
rf_model=ExtraTreesClassifier(random_state=7,n_estimators=300)  #n_estimators : 커질수록 트리 개수 늘어남(default: 100)

In [30]:
cv=RandomizedSearchCV(rf_model, param_distributions=params,n_iter=50,verbose=4)  #verbose= : 진행단계 보여줌 

In [31]:
cv.fit(x_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END criterion=gini, max_depth=14, min_samples_leaf=9;, score=0.754 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=14, min_samples_leaf=9;, score=0.754 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=14, min_samples_leaf=9;, score=0.755 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=14, min_samples_leaf=9;, score=0.754 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=14, min_samples_leaf=9;, score=0.754 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=5, min_samples_leaf=8;, score=0.754 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=5, min_samples_leaf=8;, score=0.754 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=5, min_samples_leaf=8;, score=0.755 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=5, min_samples_leaf=8;, score=0.754 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=5, min_samples_leaf=8;, score=0.754 total time= 

RandomizedSearchCV(estimator=ExtraTreesClassifier(n_estimators=300,
                                                  random_state=7),
                   n_iter=50,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': range(2, 16),
                                        'min_samples_leaf': range(5, 16)},
                   verbose=4)

In [32]:
print('[cv.best_score]',cv.best_score_)
print('[v.best_params_]',cv.best_params_)
print('[cv.best_estimator_]',cv.best_estimator_)

[cv.best_score] 0.75389649811209
[v.best_params_] {'min_samples_leaf': 9, 'max_depth': 14, 'criterion': 'gini'}
[cv.best_estimator_] ExtraTreesClassifier(max_depth=14, min_samples_leaf=9, n_estimators=300,
                     random_state=7)


In [33]:
cv_result_df=pd.DataFrame(cv.cv_results_)
cv_result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.237800,0.007913,0.025468,0.001335,9,14,gini,"{'min_samples_leaf': 9, 'max_depth': 14, 'crit...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
1,0.228453,0.002897,0.023898,0.001053,8,5,gini,"{'min_samples_leaf': 8, 'max_depth': 5, 'crite...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
2,0.230547,0.004102,0.024769,0.001138,11,6,gini,"{'min_samples_leaf': 11, 'max_depth': 6, 'crit...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
3,0.246902,0.006813,0.026323,0.001815,6,11,gini,"{'min_samples_leaf': 6, 'max_depth': 11, 'crit...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
4,0.238921,0.002498,0.025731,0.001515,9,9,log_loss,"{'min_samples_leaf': 9, 'max_depth': 9, 'crite...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
5,0.242522,0.013493,0.025383,0.001398,12,11,log_loss,"{'min_samples_leaf': 12, 'max_depth': 11, 'cri...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
6,0.265958,0.009144,0.028157,0.001662,5,15,gini,"{'min_samples_leaf': 5, 'max_depth': 15, 'crit...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
7,0.243606,0.004629,0.024548,0.001203,15,14,entropy,"{'min_samples_leaf': 15, 'max_depth': 14, 'cri...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
8,0.237427,0.006861,0.025007,0.000912,14,8,gini,"{'min_samples_leaf': 14, 'max_depth': 8, 'crit...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
9,0.245454,0.006302,0.026318,0.001420,6,9,gini,"{'min_samples_leaf': 6, 'max_depth': 9, 'crite...",0.753846,0.753846,0.754572,0.753609,0.753609,0.753896,0.000354,1
